<!--
 * @Author: Chuyang Su cs4570@columbia.edu
 * @Date: 2025-12-01 11:19:48
 * @LastEditors: Schuyn 98257102+Schuyn@users.noreply.github.com
 * @LastEditTime: 2025-12-01 11:20:08
 * @FilePath: /Unsupervised-Machine-Learning-Final-Project/Code/EDA.ipynb
 * @Description: 
 * 不要为了努力道歉
-->
## Exploratory Data Analysis

This part to fully explore the dataset and visualize the data distribution, feature relationships, and any patterns or anomalies present in the data. The goal is to gain insights that will inform subsequent modeling decisions.

In [2]:
'''
Author: Chuyang Su cs4570@columbia.edu
Date: 2025-12-01 11:19:48
LastEditors: Schuyn 98257102+Schuyn@users.noreply.github.com
LastEditTime: 2025-12-01 11:25:00
FilePath: /Unsupervised-Machine-Learning-Final-Project/Code/EDA.ipynb
Description: 
    EDA
'''
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings('ignore')

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

### Data Overview

In [5]:
df = pd.read_csv('Data/nbaplayersdraft.csv')

print(df.head())

   id  year  rank  overall_pick team          player     college  \
0   1  1989     1             1  SAC  Pervis Ellison  Louisville   
1   2  1989     2             2  LAC     Danny Ferry        Duke   
2   3  1989     3             3  SAS    Sean Elliott     Arizona   
3   4  1989     4             4  MIA       Glen Rice    Michigan   
4   5  1989     5             5  CHH       J.R. Reid         UNC   

   years_active   games  minutes_played   points  total_rebounds  assists  \
0          11.0   474.0         11593.0   4494.0          3170.0    691.0   
1          13.0   917.0         18133.0   6439.0          2550.0   1185.0   
2          12.0   742.0         24502.0  10544.0          3204.0   1897.0   
3          15.0  1000.0         34985.0  18336.0          4387.0   2097.0   
4          11.0   672.0         15370.0   5680.0          3381.0    639.0   

   field_goal_percentage  3_point_percentage  free_throw_percentage  \
0                  0.510               0.050             

从89年开始选秀，顺位从状元依次向下排列。疑点（已解明）：rank是选秀前的预测顺位overall pick是球员最终的顺位

问题：有years active，跟我们预想的不同，因此需要truncate所有这一列不满5年的球员，将其作为测试集。

有出场次数、出场总时长、总得分等生涯总数，以及生涯平均命中率，之后的场均数据可以由之前的数据计算得出。

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1922 entries, 0 to 1921
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         1922 non-null   int64  
 1   year                       1922 non-null   int64  
 2   rank                       1922 non-null   int64  
 3   overall_pick               1922 non-null   int64  
 4   team                       1922 non-null   object 
 5   player                     1922 non-null   object 
 6   college                    1585 non-null   object 
 7   years_active               1669 non-null   float64
 8   games                      1669 non-null   float64
 9   minutes_played             1669 non-null   float64
 10  points                     1669 non-null   float64
 11  total_rebounds             1669 non-null   float64
 12  assists                    1669 non-null   float64
 13  field_goal_percentage      1665 non-null   float

总共1922个人，存在不少缺失值，数据类型多样。

结合总体的数据来看，似乎有不少球员是只有选秀顺位没有任何其他数据的，这部分应该清除掉，条件暂定为year active及之后全部为空。

最后四个数据：WS、WS/48 是基于 box score + 球队效率的公式计算；
BPM、VORP 是用回归模型拟合出来的估计指标。

它们都不属于原始数据（raw stats），而是二次加工后的 advanced metrics。这点需要注意。

结合其他人的数据分析结果，不同年份选秀数量有些许差异，或许应该注意这点。

想法转变：被选秀选中但是从未出现在球场或很少出现在球场上的球员应该作为考虑因素，这样可以保留那些低顺位的球员难以出场的信息。

另外，我希望可以选择total data作为分析数据，因为其与场均数据只能选其一，而我更希望数据之间保持类型统一，这样方便处理，不必转换类型。场均数据可以作为之后结果的展示。

一个问题：能否将这1922个人全都放到一张2d图上进行展示？会不会过于密集？

综上所述：将数据进行切分，将2021年的那些尚在新秀赛季的球员作为后续试验的验证组；将场均数据，高级数据（后面四个）和球员姓名分离出模型训练部分，仅作展示用；将球员的选秀年份、被选中的队伍以及出身大学以embedding的方式转换类型；drop掉id的部分。将最终剩下的部分进行一个2d的pca，然后把高端数据做一个分段，每个画一张图，用四分段的方式进行染色。

In [7]:
df.describe()

,id,year,rank,overall_pick,years_active,games,minutes_played,points,total_rebounds,assists,field_goal_percentage,3_point_percentage,free_throw_percentage,average_minutes_played,points_per_game,average_total_rebounds,average_assists,win_shares,win_shares_per_48_minutes,box_plus_minus,value_over_replacement
count,1922.000000,1922.000000,1922.000000,1922.000000,1669.000000,1669.000000,1669.000000,1669.000000,1669.000000,1669.000000,1665.000000,1545.000000,1633.000000,1669.000000,1669.000000,1669.000000,1669.000000,1669.000000,1668.000000,1668.000000,1669.000000
mean,961.500000,2005.317378,29.694589,29.694589,6.332534,348.042540,8399.055722,3580.413421,1497.009587,774.300779,0.436568,0.272405,0.716825,18.134032,7.275734,3.194368,1.550749,17.873697,0.061691,-2.311271,4.403176
std,554.977927,9.456946,16.912454,16.912454,4.656321,324.897567,9845.871529,4826.142847,2003.686388,1284.602969,0.083846,0.128339,0.118702,8.707656,4.969343,2.083895,1.488536,27.989805,0.094467,4.143403,11.461729
min,1.000000,1989.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.700000,-1.264000,-52.000000,-8.500000
25%,481.250000,1997.000000,15.000000,15.000000,2.000000,72.000000,838.000000,265.000000,128.000000,46.000000,0.404000,0.222000,0.659000,11.000000,3.400000,1.700000,0.500000,0.400000,0.030000,-3.900000,-0.400000
50%,961.500000,2005.000000,30.000000,30.000000,5.000000,235.000000,4204.000000,1552.000000,656.000000,257.000000,0.435000,0.317000,0.736000,17.700000,6.200000,2.800000,1.100000,5.300000,0.069000,-2.000000,0.000000
75%,1441.750000,2013.000000,44.000000,44.000000,10.000000,584.000000,13246.000000,5150.000000,2139.000000,910.000000,0.474000,0.356000,0.797000,24.800000,10.000000,4.200000,2.100000,24.500000,0.104000,-0.300000,4.500000
max,1922.000000,2021.000000,60.000000,60.000000,22.000000,1541.000000,52139.000000,37062.000000,15091.000000,12091.000000,1.000000,1.000000,1.000000,41.100000,27.200000,13.300000,9.500000,249.500000,1.442000,51.100000,142.600000


有一些数据的均值方差等指标是有意义的，然而前面几个并没有。

#### 可调整部分：

选择性的进行one hot，年份、球队和大学都可以不要。

是否标准化？

是否加入没有上过场的球员进入模型？

可以加入场均数据的百分位水平，以展示球员在对应维度的排位水平。

pca or nmf？

讲稿

structure：
数据
    概述
        球员信息
        box score
        advanced metrics（重点）
            ws + ws48：衡量球员能带给球队多少胜场的能力，偏重于宏观
                ws由进攻ws（ows）和防守ws（dws）两部分组成
                ws48则是在球员ws的基础上除以上场时间然后乘上48，代表球员以这个效率打满48分钟也就是全场能给球队带来多少胜场
            bpm + vorp：衡量球员上场能让球队赢对手多少分，偏重于每一场内的表现
                bpm就是一个回归模型估计的球员每百回合对球队净胜分的贡献，由于其包含了全部的可见的box score，因此对于攻防一体的球员会有较强的偏好
                而vorp则是简单的对于bpm的一个变换，通过一个常数-2来衡量球员在上场后给球队带来的净胜分的变化。
    处理
        数据清洗：经过初步分析决定删除从未上过场的球员，并对剩余的球员进行缺失值的补全，所有的数值型变量的缺失值全都填为0，而college一栏基于现实情况将缺失值填为International；由于数据中的advanced metrics存在与box score的联系，为了防止模型的过拟合以及为了展示时作为参考，因此选择在pca和umap阶段drop这部分；由于在本数据中存在场均数据和总数据，而前者是从后者处直接计算得来，因此显然共线，在二选一的情况下为了模型保持量纲的统一选择后者。
        fe：首先收集了大学的情况，最终选择出nba球员最多的前20个（包括interational）进行独热编码并将其余所有编码为others，对选修年进行标准化，对选秀队伍进行独热编码，但是由于pca和umap的结果不好，因此选择全部弃掉，最终只编码选择的数字部分，也就是：games minutes_played points total_rebounds assists field_goal_percentage 3_point_percentage free_throw_percentage。
    dimension reduction：
        pca：首先绘制了累计解释方差的曲线，以及每个主成分解释方差水平的柱状图，此外还输出了前十个主成分的组成成分，在此处展示前两个主成分，这两个加起来解释了74%的方差，可以看到第一个主成分更看重除了命中率以外的部分，在这个方向数字越大代表一个球员的绝对数据越高，而第二个主成分则完全相反，极度关注三分命中率和罚球命中率，而其余数据几乎都不关注乃至是负向作用。
        如之前所说，由于对非box score部分进行pca的结果糟糕，因此仅展示采用数字部分的版本。我们在2d的空间绘制了四张pca的散点图，分别以4个高级指标的4分位水平进行染色，在这个指标上表现最差一档的球员会被染成红色，而表现最好的会被染成深绿色，而从红色到深绿色的过渡代表了球员水平在这项指标上的提升，并在其上高亮了六名著名球星。可以看到詹姆斯 邓肯 科比这些老资历的球员在pc1上的位置十分靠前，而库里 欧文这两个现代后卫则在pc2上非常靠前，在同等的pc2水平上，库里在pc1的位置要高于欧文，这也符合现实。
        抛开这些球星，从宏观来看，可以看到我们的数据在win shares上形成了十分形状良好的簇，也就是沿着pc1的方向越来越强，这符合直觉，因为在场上绝对数据越多的球员更有可能带领球队拿到胜利，而且越沿着pc1的方向前进，实际上点就越发的稀疏，这也体现出了NBA的一个特点，即出色的球员可以遥遥甩开其余的平庸的大多数。相对而言几乎只是在ws上做了些许改动的ws48在我们的图中却是簇的形状最不好的，除了红色的点相对而言比较集中以外，其余的点都混杂在一起，这说明即便绝对数据相对较少的球员也能有效地为球队的获胜做出贡献，这也符合常识，因为一个球队仅可能有一到三个超级巨星可以有大量数据进账，而所有参赛的球员都会获得相同的胜利，这也解释了篮球的一个底层逻辑，往往不是只要堆球星就能获得所有的胜利。其实在全部四个图片我们都可以发现一个共性，那就是虽然所有其他的区域都会变化，但是只有红色区域的那一部分人是不会有太大变动的。其实在我们没有删掉从未上过场的busts的时候，这个特征就已经十分稳固，然而即便我们收紧了约束，我们会发现这些在最后25%的球员永远都是在最后，而且在pc2上的分布十分广，这说明这些球员其实各不相同，唯一的共性就是绝对数据太少，这其中存在着彼此联系的因果关系，由于数据量的缺乏，我们在此无法做进一步的讨论，但是是一个有趣的点。刨除这些吊车尾，我们可以看到在bpm这个图中，除了在pc1上足够远的球员以外，实际上数据是沿着pc2的方向分层的，也就是说除了那些绝对数据非常高的球员以外，其实能获得更高bpm的是三分命中率和罚球命中率足够高的球员。vorp的结果应该是最有趣的，因为我们第一次发现在pc1最近的位置居然是50%分位的球员而不是25%，而25%分位的球员居然会出现在pc1比较远的位置，这个结果代表有些球员虽然绝对数据够高，但是其实球队在没有他们的时候能赢更多分，而那些绝对数据最少的球员，其实并不是上场后最伤害球队的。
